In [2]:
import pandas as pd
import itertools

Option 1 is to pick a column as a list and do the trick with standard python list tools

In [20]:
#a dummy version to play with very small sets is
data = [['tom', "1"], ['jason', "13"], ['juli', 3], ['juli', 3], ['louise', 3] , 
        ['peter', 4],['mark', 10], ['nick', 2], ['andrea', 14], ['james', 0], ['john', ''], ['linn', 'nan'],['simon', "error"]  ] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['name', 'customer_id']) 
 
field_to_check='customer_id'
df[field_to_check]=pd.to_numeric(df[field_to_check], errors='coerce')
#An alternative  method is if they are all strings, it is faster:
#df[field_to_check]= df[df[field_to_check].apply(lambda x: x.isnumeric())]

df.fillna(0,inplace=True)
df[field_to_check]=df[field_to_check].astype(int)
df[field_to_check]

0      1
1     13
2      3
3      3
4      3
5      4
6     10
7      2
8     14
9      0
10     0
11     0
12     0
Name: customer_id, dtype: int32

We add here a quick check for duplicates, it is not essential for this recipe, but it is important you identify these before doing sequence checks

In [33]:
duplicated_rows = df[df.duplicated([field_to_check],keep='first')]
print("Duplicates found: ",len(duplicated_rows))
duplicated_rows


Duplicates 0


,Unnamed: 0,customer_id,gender,first_name,last_name,dob,address,city,country


and we drop them, notice the "inplace" which means the data frame df will be changed, rather than having to do  
df=df.drop_duplicates...  
or  
df_deduped= df.drop_duplicates...


In [22]:
df.drop_duplicates(subset=[field_to_check],keep='first', inplace=True)

Now is where we do the actual test for numerical sequence

In [23]:
list_to_check= df[field_to_check].tolist()
# here the key command is "set" which picks only unique values.
# note that this means that we dont check for duplicates or
# non numeric, blanks etc, this should be done beforehand

list_to_check=sorted(set(list_to_check))
#list_to_check.sort()
gap_list=[]
n= 0
for el in list_to_check:
    if n==0:
        n= el-1    
    if el-n > 1:
        gap_list.extend(range(n+1,el))
    n=el
gap_list

[5, 6, 7, 8, 9, 11, 12]

this optional routine converts it to pair of ranges, which can be useful if the gaps are clustered

In [24]:
def to_ranges(iterable):
    iterable = sorted(set(iterable))
    for key, group in itertools.groupby(enumerate(iterable), lambda t: t[1] - t[0]):
        group = list(group)
        yield group[0][1], group[-1][1]
list(to_ranges(gap_list))

[(5, 9), (11, 12)]

Option 2 is to do everything with panda data frames, iterating through each record, having previously sorted by the target column/field

In [34]:
df.sort_values(by=[field_to_check], inplace=True)
gap_list=[]
n= 0
for index, row in df.iterrows():
    if n==0:
        n= row[field_to_check]-1    
    if (row[field_to_check]-n) > 1:
        gap_list.extend(range(n+1,row[field_to_check]))
    n=row[field_to_check]
gap_list

[2,
 4,
 23,
 25,
 80,
 89,
 143,
 217,
 367,
 419,
 431,
 455,
 493,
 606,
 624,
 635,
 684,
 703,
 875,
 882]

In [35]:
df2 = pd.DataFrame({'id':gap_list})
df2.to_excel("./output/Sequence Gaps.xlsx")
print(len(gap_list))
df2

20


,id
0,2
1,4
2,23
3,25
4,80
5,89
6,143
7,217
8,367
9,419


All done!!!   
Now using the data file it is simple

In [37]:
df=pd.read_excel('./test_data/data_customer.xlsx')
print(df.shape) # number of rows and columns
df.head() # first five records

(980, 9)


,Unnamed: 0,customer_id,gender,first_name,last_name,dob,address,city,country
0,0,1,F,Lindsey,Baldwin,1992-07-11,656 Mark Coves Apt. 230,Goodwinton,Grenada
1,1,3,F,Lorraine,Stone,2015-04-09,050 Ashley Burg,Garyton,Croatia
2,2,5,F,Natalie,Dean,2003-06-28,321 Barbara Creek,Mallorybury,Bouvet Island (Bouvetoya)
3,3,6,F,Stacey,Moon,2000-09-08,45842 Jacob Ports Apt. 879,Lake Michelleside,Eritrea
4,4,7,M,Ivan,Young,2006-01-21,193 Susan Wall Apt. 899,Lucasstad,Mayotte


In [39]:
field_to_check="customer_id"
df.sort_values(by=[field_to_check], inplace=True)
gap_list=[]
n= 0
for index, row in df.iterrows():
    if n==0:
        n= row[field_to_check]-1    
    if (row[field_to_check]-n) > 1:
        gap_list.extend(range(n+1,row[field_to_check]))
    n=row[field_to_check]
print("Missing values:"+str(len(gap_list)))
gap_list

Missing values:20


[2,
 4,
 23,
 25,
 80,
 89,
 143,
 217,
 367,
 419,
 431,
 455,
 493,
 606,
 624,
 635,
 684,
 703,
 875,
 882]